In [34]:
import tensorflow as tf

In [35]:
class residual1(tf.keras.Model):
    def __init__(self,channels,_1x1conv=False,strides=1):
        super().__init__()
        self.conv1=tf.keras.layers.Conv2D(channels,padding='same',kernel_size=3,strides=strides)
        self.conv2=tf.keras.layers.Conv2D(channels,padding='same',kernel_size=3)
        self.conv3=None
        if _1x1conv:
            self.conv3=tf.keras.layers.Conv2D(channels,kernel_size=1,strides=strides)
        self.bn1=tf.keras.layers.BatchNormalization()
        self.bn2=tf.keras.layers.BatchNormalization()

    def call(self,x):
        y=tf.keras.activations.relu(self.bn1(self.conv1(x)))
        y=self.bn2(self.conv2(y))
        if self.conv3 is not None:
            x=self.conv3(x)
        y+=x
        return tf.keras.activations.relu(y)


In [36]:
# blk = residual(3)
# x = tf.random.normal((4, 6, 6, 3))
# y = blk(x)
# y.shape

In [37]:
# blk = residual(6, _1x1conv=True, strides=2)
# blk(x).shape

In [38]:
class ResNet(tf.keras.Model):
    def __init__(self,dimensions,lr=0.1,num_classes=10):
        super(ResNet,self).__init__()
        # self.save_hyperparameters()
        self.net=tf.keras.models.Sequential(self.block1())
        for i,n in enumerate(dimensions):
            self.net.add(self.block(*n,first_block=(i==0)))
        self.net.add(tf.keras.models.Sequential([
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(units=num_classes)
        ]))

    def block1(self):
        return tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64,kernel_size=7,strides=2,padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same')])

    def block(self,num_residuals,channels,first_block=False):
        b=tf.keras.models.Sequential()
        for i in range(num_residuals):
            if i==0 and not first_block:
                b.add(residual1(channels,_1x1conv=True,strides=2))
            else:
                b.add(residual1(channels))
        return b

In [39]:
# x=ResNet()
# x.block1()

In [40]:
class ResNet18(ResNet):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__(((2, 64), (2, 128), (2, 256), (2, 512)),
                       lr, num_classes)

ResNet18()